In [1]:
%pip install groq # Instalando o groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.2 MB/s eta 0:00:00


In [2]:
import os

os.environ['GROQ_API_KEY'] = "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" # Utilizando a chave da API que peguei no Groq

In [3]:
import os
from groq import Groq # Importa a classe que nos permite interagir com a API do Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
) # Inicializando o Groq e passando a minha chave ali de cima

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models", # Passando uma mensagem teste para o nosso modelo
        }
    ],
    model="llama3-70b-8192", # Definindo que modelo vamos utilizar
)

print(chat_completion.choices[0].message.content) # Aqui printamos a resposta que o modelo vai dar para o nosso prompt/mensagem que passamos para ele ali em cima

Fast language models are artificial intelligence (AI) models that can process and generate human-like language at high speeds, often in real-time or near real-time. The importance of fast language models lies in their ability to transform various industries and aspects of our lives, including:

1. **Customer Service Chatbots**: Fast language models enable chatbots to respond quickly and accurately to customer inquiries, providing 24/7 support and improving customer satisfaction.
2. **Virtual Assistants**: Fast language models power virtual assistants like Alexa, Google Assistant, and Siri, allowing them to understand and respond to voice commands rapidly.
3. **Language Translation**: Fast language models can translate languages in real-time, facilitating global communication and breaking language barriers.
4. **Content Generation**: Fast language models can generate high-quality content, such as articles, social media posts, and product descriptions, at an unprecedented pace, revolutio

In [4]:
class Agent:
  def __init__(self, client, system):
    self.client =  client # Inicializando o cliente
    self.system = system # Inicializando a mensagem do sistema para o modelo entender como ele deve trabalhar em loop
    self.messages = [] # Lista de mensagens que vamos passar
    if self.system is not None: # Se a mensagem não for vazia nos vamos adiciona-la na nossa lista
      self.messages.append({"role":"system", "content": self.system})

  def __call__(self, message=""):
      if message: # Vai ser ativado sempre que o nosso agente for chamado
        self.messages.append({"role":"user", "content": message}) # Se receber uma mensagem vai adiciona-la na lista
      result = self.execute() # Depois de receber a mensagem vamos executar o agente com todas as mensagens da nossa lista
      self.messages.append({"role":"assistant", "content": result}) # Depois vai adicionar o resultado na nossa lista
      return result # E por fim retornar o resultado para gente

  def execute(self): # Método de execução que usamos no método anterior após recebermos a mensagem, faz o modelo executar a nossa mensagem que enviaremos nos códigos a seguir
      completion = client.chat.completions.create(
          messages= self.messages ,
          model="llama3-70b-8192", # Definindo que modelo vamos utilizar
      )
      return completion.choices[0].message.content # Retorna a resposta que o modelo criou

In [5]:
system_prompt  = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_population:
e.g. get_population: Goiania
returns population of Goiania

Example session:

Question: What is the population of Goiânia times 2?
Thought: I need to find the population of Goiania
Action: get_population: Goiania
PAUSE

You will be called again with this:

Observation: 1555626

Thought: I need to multiply this by 2
Action: calculate: 1555626 * 2
PAUSE

You will be called again with this:

Observation: 3111252

If you have the answer, output it as the Answer.

Answer: The population of Goiânia times 2 is 3111252

Now it's your turn:
""".strip()

# Esse o prompt do sistema que vai fazer o agente entender como ele deve se comportar para seguir o loop

In [6]:
# Ferramentas
def calculate(operation):
  return eval(operation) # Ferramenta padrão de calculadora

def get_population(city) -> float: # Ferramenta que serve como uma "pseudo-pesquisa" para que o agente possa realizar o que lhe foi pedido
  match city.lower():
    case "goiania":
      return 1555626
    case "sao paulo":
      return 11895578
    case "rio de janeiro":
      return 6211223
    case "brasilia":
      return 2982818

In [12]:
ibge = Agent(client, system_prompt) # Instanciando o agente com o nome IBGE porque ele é um especialista em populações de cidades

**TESTANDO O AGENTE MANUALMENTE**

In [13]:
result = ibge("What is the sum of the population of Sao Paulo and Rio de Janeiro times 2?") # Dando um prompt teste para o agente
print(result) # Printando o resultado para vermos se está tudo de acordo

Thought: I need to find the population of Sao Paulo and Rio de Janeiro, then add them together, and finally multiply the result by 2.


In [14]:
ibge.messages # Aqui é um log das mensagens que o agente recebe e os pensamentos que ele cria

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [16]:
result = ibge() # Aqui ele vai definir que ação ele deve tomar
print(result) # Printamos para conferir se está tudo certo

Action: get_population: Sao Paulo
PAUSE


In [17]:
observation = get_population("sao paulo") # Como ainda não criamos o loop já damos a resposta da ação dele diretamente para ele
print(observation) # Printamos para conferir se está tudo certo

11895578


In [18]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt) # Dando a observação para o agente
print(result) # Printamos para conferir se está tudo certo

Thought: Now I have the population of Sao Paulo, I need to find the population of Rio de Janeiro and add it to this number.


In [19]:
ibge.messages # Conferindo o log para termos certeza que o loop está de acordo, Thought, Action e Observation.

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [21]:
result = ibge() # Esperamos a próxima ação
print(result) # Printamos para conferir se está tudo certo

Action: get_population: Rio de Janeiro
PAUSE


In [22]:
observation = get_population("rio de janeiro") # Criando a observação para ele
print (observation) # Printamos para conferir se está tudo certo

6211223


In [23]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt) # Colocando a observação nele
print(result) # Printamos para conferir se está tudo certo

Thought: Now I have the populations of both Sao Paulo and Rio de Janeiro, I need to add them together and then multiply the result by 2.


In [24]:
ibge.messages # Conferindo o log

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [26]:
result = ibge() # Repetindo o loop, agora na ação
print(result) # Printamos para conferir se está tudo certo

Action: calculate: 11895578 + 6211223
PAUSE


In [27]:
observation = calculate("(11895578 + 6211223)") # Criando a observação para ele
print(observation) # Printamos para conferir se está tudo certo

18106801


In [28]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt) # Dando a observação para ele
print(result) # Printamos para conferir se está tudo certo

Thought: Now I have the sum of the populations, I need to multiply it by 2.


In [29]:
ibge.messages # Conferindo o log

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

In [31]:
result = ibge() # Fazendo ele nos dar a próxima ação
print(result) # Printamos para conferir se está tudo certo

Action: calculate: 18106801 * 2
PAUSE


In [32]:
observation = calculate("18106801 * 2") # Criando a última observação
print(observation) # Printamos para conferir se está tudo certo

36213602


In [33]:
next_prompt = f"Observation: {observation}"
result = ibge(next_prompt) # Dando a observação para ele
print(result) # Printamos para conferir se está tudo certo

Thought: Now I have the final result, which is the sum of the population of Sao Paulo and Rio de Janeiro times 2.

Answer: The sum of the population of Sao Paulo and Rio de Janeiro times 2 is 36213602.


In [34]:
ibge.messages # Conferindo o log

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_population:\ne.g. get_population: Goiania\nreturns population of Goiania\n\nExample session:\n\nQuestion: What is the population of Goiânia times 2?\nThought: I need to find the population of Goiania\nAction: get_population: Goiania\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1555626\n\nThought: I need to multiply this by 2\nAction: calculate: 1555626 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 3111252\n\nIf you have 

**CRIANDO O LOOP PARA O AGENTE FUNCIONAR APENAS COM O PROMPT**

In [35]:
import re # Criando o loop do agente

def agent_loop(max_iterations, system, query):
  agent = Agent(client, system_prompt) # Inicilizando o agente
  tools = ['calculate', 'get_population'] # Inicilizando as ferramentas que temos
  next_prompt = query # Definimos o primeiro prompt como pedido que vamos mandar para o loop do agente
  i = 0
  while i < max_iterations: # Iniciando o loop
    i += 1
    result = agent(next_prompt) # Chamando o agente
    print(result) # Printando o resultado para análise

    if "PAUSE" in result and "Action" in result: # Se o resultado tiver PAUSE e AÇÃO
      action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE) # Nós encontramos o nome da ação e os parâmetros com essa linha de código
      chosen_tool = action[0][0] # Organizando a ferramenta selecionada
      arg = action[0][1] # Organizado o parâmetro encontrado

      if chosen_tool in tools: # Se a ferramenta estiver entre as que nós temos
        result_tool = eval(f"{chosen_tool}('{arg}')") # Usamos e ferramenta e temos o resultado dela
        next_prompt = f"Observation: {result_tool}" # Que é analisado nessa observação

      else:
        next_prompt = "Observation: Tool not found" # Se não tivermos a ferramenta vamos dizer que não encontramos a ferramenta para que o sistema procure outra correta

      print(next_prompt) # Printamos para sabermos o que está acontecendo
      continue # Continua a fazer mais iterações

    if "Answer" in result: # Se o agente tiver "resposta" no seu resultado nós paramos o loop
      break


In [36]:
agent_loop(max_iterations=50, system=system_prompt, query="What is the sum of the population of Sao Paulo, Goiania and Rio de Janeiro and all of it times 5?") # Implementando o loop do agente criado

Thought: I need to find the population of Sao Paulo, Goiania, and Rio de Janeiro, and then add them together.
Action: get_population: Sao Paulo
PAUSE
Observation: 11895578
Thought: I have the population of Sao Paulo, now I need to find the population of Goiania and Rio de Janeiro.
Action: get_population: Goiania
PAUSE
Observation: 1555626
Thought: I have the population of Sao Paulo and Goiania, now I need to find the population of Rio de Janeiro.
Action: get_population: Rio de Janeiro
PAUSE
Observation: 6211223
Thought: I have the population of Sao Paulo, Goiania, and Rio de Janeiro, now I need to add them together.
Action: calculate: 11895578 + 1555626 + 6211223
PAUSE
Observation: 19662427
Thought: I have the sum of the populations, now I need to multiply it by 5.
Action: calculate: 19662427 * 5
PAUSE
Observation: 98312135
Answer: The sum of the population of Sao Paulo, Goiania and Rio de Janeiro and all of it times 5 is 98312135.
